In [1]:
import os, warnings
from pathlib import Path

import numpy as np
import dynamo as dyn
import spateo as st
warnings.filterwarnings('ignore')

/home/pc001/anaconda3/envs/BioProjects/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/pc001/anaconda3/envs/BioProjects/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/pc001/anaconda3/envs/BioProjects/lib/python3.8/site-packag

In [2]:
from typing import Optional, Tuple, Union

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pyvista import PolyData, UnstructuredGrid

try:
    from typing import Literal
except ImportError:
    from typing_extensions import Literal


def add_model_labels(
        model: Union[PolyData, UnstructuredGrid],
        labels: np.ndarray,
        key_added: str = "groups",
        where: Literal["point_data", "cell_data"] = "cell_data",
        colormap: Union[str, list, dict, np.ndarray] = "rainbow",
        alphamap: Union[float, list, dict, np.ndarray] = 1.0,
        mask_color: Optional[str] = "gainsboro",
        mask_alpha: Optional[float] = 0.0,
        inplace: bool = False,
) -> Tuple[Optional[PolyData or UnstructuredGrid], Optional[Union[str]]]:
    model = model.copy() if not inplace else model
    labels = np.asarray(labels).flatten()

    cu_arr = np.sort(np.unique(labels), axis=0).astype(object)
    raw_labels_hex = labels.copy().astype(object)
    raw_labels_alpha = labels.copy().astype(object)
    raw_labels_hex[raw_labels_hex == "mask"] = mpl.colors.to_hex(mask_color)
    raw_labels_alpha[raw_labels_alpha == "mask"] = mask_alpha

    # Set raw hex.
    if isinstance(colormap, str):
        if colormap in list(mpl.colormaps()):
            lscmap = mpl.cm.get_cmap(colormap)
            raw_hex_list = [mpl.colors.to_hex(lscmap(i)) for i in np.linspace(0, 1, len(cu_arr))]
            for label, color in zip(cu_arr, raw_hex_list):
                raw_labels_hex[raw_labels_hex == label] = color
        else:
            raw_labels_hex[raw_labels_hex != "mask"] = mpl.colors.to_hex(colormap)
    elif isinstance(colormap, dict):
        for label, color in colormap.items():
            raw_labels_hex[raw_labels_hex == label] = mpl.colors.to_hex(color)
    elif isinstance(colormap, list) or isinstance(colormap, np.ndarray):
        raw_hex_list = np.array([mpl.colors.to_hex(color) for color in colormap]).astype(object)
        for label, color in zip(cu_arr, raw_hex_list):
            raw_labels_hex[raw_labels_hex == label] = color
    else:
        raise ValueError("`colormap` value is wrong." "\nAvailable `colormap` types are: `str`, `list` and `dict`.")

    # Set raw alpha.
    if isinstance(alphamap, float) or isinstance(alphamap, int):
        raw_labels_alpha[raw_labels_alpha != "mask"] = alphamap
    elif isinstance(alphamap, dict):
        for label, alpha in alphamap.items():
            raw_labels_alpha[raw_labels_alpha == label] = alpha
    elif isinstance(alphamap, list) or isinstance(alphamap, np.ndarray):
        raw_labels_alpha = np.asarray(alphamap).astype(object)
    else:
        raise ValueError(
            "`alphamap` value is wrong." "\nAvailable `alphamap` types are: `float`, `list` and `dict`."
        )

    # Set rgba.
    labels_rgba = [mpl.colors.to_rgba(c, alpha=a) for c, a in zip(raw_labels_hex, raw_labels_alpha)]
    labels_rgba = np.array(labels_rgba).astype(np.float32)

    # Added rgba of the labels.
    if where == "point_data":
        model.point_data[f"{key_added}_rgba"] = labels_rgba
    else:
        model.cell_data[f"{key_added}_rgba"] = labels_rgba

    plot_cmap = None

    # Added labels.
    if where == "point_data":
        model.point_data[key_added] = labels
    else:
        model.cell_data[key_added] = labels

    return model if not inplace else None, plot_cmap

In [3]:
sample_id = "E7_8h"
os.chdir(f"/media/pc001/Yao/Projects/Project_spateo/mouse_heart/figure6")
cpo = [(41, 1209, 57), (13, 8, -3), (0, 0, 1)]

out_h5ad_path = f"droso_{sample_id}_midgut_morpho/h5ad"
out_image_path = f"droso_{sample_id}_midgut_morpho/image/midgut_curl"
Path(out_h5ad_path).mkdir(parents=True, exist_ok=True)
Path(out_image_path).mkdir(parents=True, exist_ok=True)

In [4]:
# Read anndata
key = "morpho_curl"
adata = st.read_h5ad(os.path.join(out_h5ad_path, f"droso_{sample_id}_midgut_v4.h5ad"))
adata = adata[:, ~adata.var.index.str.startswith(f"mt:")]

import scanpy as sc
sc.pp.normalize_total(adata, target_sum=2e3)
sc.pp.log1p(adata)

pc_model = st.tdr.read_model(os.path.join(out_h5ad_path, "midgut_morphometrics_pc_model.vtk"))
trajectory_model = st.tdr.read_model(os.path.join(out_h5ad_path, "midgut_morphometrics_trajectory_model.vtk"))
adata

AnnData object with n_obs × n_vars = 2159 × 7595
    obs: 'area', 'slices', 'anno_cell_type', 'anno_tissue', 'anno_germ_layer', 'actual_stage', 'scc', 'anno_tissue_new', 'V_z', 'anno_germ_layer_new', 'backbone_nodes', 'backbone_scc', 'segments', 'V_x', 'morpho_acceleration', 'morpho_curvature', 'morpho_curl', 'morpho_torsion', 'morpho_divergence'
    uns: 'VecFld_morpho', 'pp', 'log1p'
    obsm: '2d_realign_spatial', '3d_align_spatial', 'V_cells_mapping', 'X_cells_mapping', 'X_pca', 'X_umap', 'morpho_acceleration', 'morpho_curl', 'morpho_curvature', 'morpho_velocity', 'raw_spatial', 'tdr_spatial'
    layers: 'X_counts', 'spliced', 'unspliced'

## GLM DEGs

In [5]:
selected_increased_genes = ["Npc2g", "CG2260", "CG33099", "lncRNA:CR43334", "scf", "nmo", "Tom", "CG12522", "Snp", "CG34383"]
selected_decreased_genes = ["Kaz-m1", "zip", "Pka-C3", "CG15152", "CG6770", "Trx-2", "peb", "CG14715", "CG8005", "Tep4"]
selected_genes = selected_increased_genes + selected_decreased_genes

In [7]:
for gn in selected_genes:# list(glm_data.index):
    _gn = str(gn).replace(":", "_") if ":" in gn else gn
    
    pc_index=pc_model.point_data["obs_index"].tolist()
    exp = adata[pc_index, gn].X.A.flatten()
    exp[exp <= 0] = 0
    
    amap = exp.copy()
    amap = amap / np.max(amap)
    amap[amap <= 0.2] = 0.2
    add_model_labels(model=pc_model, labels=exp, key_added=gn, alphamap=amap, where="point_data",inplace=True, colormap="Reds")
    
    st.pl.three_d_plot(
        model=pc_model,
        key=gn,
        model_style="points",
        model_size=16,
        opacity=1,
        colormap=None,
        show_legend=True,
        jupyter=False,
        off_screen=True,
        cpo=cpo,
        window_size=(2560, 2048),
        text=gn,
        filename=os.path.join(out_image_path, f"midgut_{key}_glm_degs_genes_{_gn}.pdf")
    )

In [6]:
for gn in ["zip"]:
    interpolated_gp_adata = st.tdr.gp_interpolation(source_adata=adata.copy(), spatial_key="3d_align_spatial", keys=gn,
                                                    target_points=np.asarray(pc_model.points), device="0",
                                                    training_iter=100)
    interpolated_gp_pc, _ = st.tdr.construct_pc(adata=interpolated_gp_adata.copy(), spatial_key="3d_align_spatial", groupby=gn,
                                                key_added=gn)
    _gn = str(gn).replace(":", "_") if ":" in gn else gn
    
    st.pl.three_d_plot(
        model=interpolated_gp_pc,
        key=gn,
        model_style="points",
        model_size=16,
        opacity=0.6,
        colormap="Reds",
        show_legend=True,
        jupyter=False,
        off_screen=True,
        cpo=cpo,
        window_size=(2560, 2048),
        text=gn,
        filename=os.path.join(out_image_path, f"midgut_{key}_glm_degs_interpolated_genes_{_gn}.pdf")
    )

{'obs_keys': [], 'var_keys': ['zip']}


Minibatch: 100%|██████████| 3/3 [00:00<00:00, 76.46it/s, loss=1.31]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 78.68it/s, loss=1.1]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 87.82it/s, loss=1.19]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 74.17it/s, loss=0.973]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 86.05it/s, loss=0.988]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 87.93it/s, loss=0.947]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 93.99it/s, loss=1.03]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 89.58it/s, loss=0.912]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 75.47it/s, loss=0.922]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 87.98it/s, loss=0.824]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 80.81it/s, loss=0.817]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 76.65it/s, loss=0.903]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 69.27it/s, loss=0.736]

Minibatch: 100%|██████████| 3/3 [00:00<00:00, 74.82it/s, loss=0.774]

Minibatch: 100%|█████████

|-----> Creating an adata object with the interpolated expression...
|-----> [GaussianProcessInterpolation] in progress: 100.0000%
|-----> [GaussianProcessInterpolation] finished [10.8998s]
